In [1]:
import argparse
import os
import shutil

In [7]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings

In [9]:
CHROMA_PATH = "chroma_db"
DATA_PATH = "data"

In [29]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--reset", action="store_true", help="Reset the database.")
    args = parser.parse_args(args=[])
    if args.reset:
        print("✨ Clearing Database")
        clear_database()

    documents = load_documents()
    chunks = split_documents(documents)
    add_to_chroma(chunks)

In [13]:
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)
    return document_loader.load()


In [15]:
def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)


In [58]:
import time
from langchain.docstore.document import Document

def add_to_chroma(chunks: list[Document], batch_size=50, delay_seconds=1):
    embedding_function = CohereEmbeddings(model="embed-english-light-v2.0", cohere_api_key=API)

    db = Chroma(
        persist_directory=CHROMA_PATH,
        embedding_function=embedding_function
    )

    chunks_with_ids = calculate_chunk_ids(chunks)

    existing_items = db.get(include=[])
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    new_chunks = [chunk for chunk in chunks_with_ids if chunk.metadata["id"] not in existing_ids]

    if not new_chunks:
        print("✅ No new documents to add")
        return

    print(f"👉 Adding new documents: {len(new_chunks)}")

    # Add documents in batches to avoid hitting rate limits
    for i in range(0, len(new_chunks), batch_size):
        batch = new_chunks[i:i+batch_size]
        batch_ids = [chunk.metadata["id"] for chunk in batch]
        try:
            db.add_documents(batch, ids=batch_ids)
        except Exception as e:
            print(f"Error adding batch {i // batch_size + 1}: {e}")
            # Optionally, add retry logic here

        print(f"Added batch {i // batch_size + 1} of {((len(new_chunks)-1)//batch_size)+1}")
        time.sleep(delay_seconds)  # Delay between batches to avoid rate limit



In [60]:
def calculate_chunk_ids(chunks):
    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        chunk.metadata["id"] = chunk_id

    return chunks

In [62]:
def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)


In [64]:
main()

PdfReadError("Invalid Elementary Object starting with b'p' @7333107: b'codemantra.com)/The process that creates this PDF constitutes a trade secret of '")


Number of existing documents in DB: 0
👉 Adding new documents: 15129
Added batch 1 of 303
Added batch 2 of 303
Added batch 3 of 303
Added batch 4 of 303
Added batch 5 of 303
Added batch 6 of 303
Added batch 7 of 303
Added batch 8 of 303
Added batch 9 of 303
Added batch 10 of 303
Added batch 11 of 303


Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised TooManyRequestsError: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': '754f45e24583b12b03609b6188954ec9', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '88', 'date': 'Fri, 29 Aug 2025 17:22:39 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '46', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': '2bfbea42-4e51-4f13-881e-77a7bcdcdf32', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}.
Retrying langchain_

Error adding batch 12: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': '0fdfced42c08875aa3cfd6dcfeafd073', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '86', 'date': 'Fri, 29 Aug 2025 17:22:48 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '32', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': '262fa45d-94c9-49b2-bcf9-d77bd8cddc43', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Added batch 12 of 303


Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised TooManyRequestsError: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': 'a15e6b0d95f8d215321176db2b7c7373', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '85', 'date': 'Fri, 29 Aug 2025 17:22:49 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '69', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': 'bf33f083-2f40-481f-9a8e-9c25ab837699', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}.
Retrying langchain_

...

Error adding batch 296: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': '3c4042ff62e56662b9f871266031d705', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '80', 'date': 'Fri, 29 Aug 2025 17:43:41 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '48', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': 'b0a16596-1350-4a95-94e3-4389a8a5cb4d', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Added batch 296 of 303


Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised TooManyRequestsError: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': 'e5068f5ad1e1387ac99687e1e309b3a0', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '80', 'date': 'Fri, 29 Aug 2025 17:43:43 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '43', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': '8da92360-b03f-40af-b2f8-0d38fbf1a634', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}.
Retrying langchain_

Error adding batch 297: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': '965f7e3b8b04bf43421b936e381a5e58', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '81', 'date': 'Fri, 29 Aug 2025 17:43:53 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '36', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': '8fb1e1af-d2c2-42bb-9a22-d063bf1348e0', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Added batch 297 of 303


Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised TooManyRequestsError: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': '7f55986b104e781c17b660eda7d6bdba', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '80', 'date': 'Fri, 29 Aug 2025 17:43:55 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '43', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': '05e9aad7-6425-4bb8-b8b2-47dbd68428f0', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}.


Added batch 298 of 303
Added batch 299 of 303
Added batch 300 of 303
Added batch 301 of 303
Added batch 302 of 303
Added batch 303 of 303
